<a href="https://colab.research.google.com/github/Danpollak/Word2Note/blob/master/Midi_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
!pip install MIDIUtil

     |████████████████████████████████| 1.0MB 2.7MB/s 
  Created wheel for MIDIUtil: filename=MIDIUtil-1.2.1-cp36-none-any.whl size=54571 sha256=d17b3791a69a07b368e2f3243fdd2ce63df56251338a44f1c9015e584f12af8d
  Stored in directory: /root/.cache/pip/wheels/70/f1/24/97bde012f64820632e1e5e2935df19dfbcf9e058b0734b57cd
Successfully built MIDIUtil


In [2]:
from midiutil import MIDIFile

## Note to MIDI dictionary

In [3]:
note2midi = {
    'C3':48,
    'C#3':49,
    'D3':50,
    'D#3':51,
    'E3':52,
    'F3':53,
    'F#3':54,
    'G3':55,
    'G#3':56,
    'A3':57,
    'A#3':58,
    'B3':59,
    'C4':60,
    'C#4':61,
    'D4':62,
    'D#4':63,
    'E4':64,
    'F4':65,
    'F#4':66,
    'G4':67,
    'G#4':68,
    'A4':69,
    'A#4':70,
    'B4':71,
    'C5':72,
    'C#5':73,
    'D5':74,
    'D#5':75,
    'E5':76,
    'F5':77,
    'F#5':78,
    'G5':79,
    'G#5':80,
    'A5':81,
    'A#5':82,
    'B5':83,
    'C6':84
}

In [45]:
def createMIDI(midi_data, output_midi_name,BPM=60):

    track    = 0
    channel  = 0
    time     = 0    # In beats
    tempo    = BPM  # In BPM
    volume   = 100  # 0-127, as per the MIDI standard

    MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
                          # automatically)
    MyMIDI.addTempo(track, time, tempo)

    for i, pitch in enumerate(midi_data):
      degree, duration = pitch
      if degree != 'BREAK':
        MyMIDI.addNote(track, channel, degree, time , duration, volume)
      time+=duration

    with open(output_midi_name, "wb") as output_file:
        MyMIDI.writeFile(output_file)

In [ ]:
def time2beat(tempo,time):
  return time / tempo

In [42]:
def zip_note_line(line, with_breaks=False):
  # Remove SOS
  line = line[1:]
  # Remove EOS
  line= line[:line.index('EOS')]
  zipped_line = []
  last_note = ''
  count = 0
  for note in line:
    if note.endswith('_S'):
      if count > 0:
        zipped_line.append((note2midi[last_note], count))
        count = 0
      count +=1
      last_note = note[:-2]
    elif note == 'BREAK':
      if with_breaks:
        zipped_line.append(('BREAK', 1))
        count=0
      else:
        count +=1
      zipped_line.append((note2midi[last_note], count))
      count = 0
    else:
      count+=1
  if count > 0:
    zipped_line.append((note2midi[last_note], count))


  return zipped_line

#Write note line

In [43]:
note_line = ['SOS','C3_S','D3_S','E3_S','F3_S','G3_S','EOS']

## Convert line to MIDI

In [46]:
line = zip_note_line(note_line)
BPM = 120

createMIDI(line, 'losing_my_religion.mid', BPM=BPM)